# Install

In [ ]:
!pip3 install soynlp

In [ ]:
!pip3 install pshmodule

In [ ]:
!pip3 install pickle5

In [ ]:
!pip3 install pandas==1.5.0

In [ ]:
!pip3 install swifter

In [ ]:
!curl -s https://raw.githubusercontent.com/teddylee777/machine-learning/master/99-Misc/01-Colab/mecab-colab.sh | bash

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Data Load

In [ ]:
import sys
sys.path.append('drive/MyDrive/MemeProject/src/doc2vec/')
print(sys.path)

['/content', '/env/python', '/usr/lib/python38.zip', '/usr/lib/python3.8', '/usr/lib/python3.8/lib-dynload', '', '/usr/local/lib/python3.8/dist-packages', '/usr/lib/python3/dist-packages', '/usr/local/lib/python3.8/dist-packages/IPython/extensions', '/root/.ipython', 'drive/MyDrive/MemeProject/src/doc2vec/']


In [ ]:
import re
import swifter
import config as cfg
from pshmodule.utils import filemanager as fm

In [ ]:
df = fm.load(cfg.temp_ref_data_colab)

extension : .pickle
Loaded 208700 records from drive/MyDrive/MemeProject/data/temp_for_ref.pickle


In [ ]:
df = df[['u']]

In [ ]:
df.drop_duplicates(inplace=True)

In [ ]:
df.rename(columns={'u':'content'}, inplace=True)

In [ ]:
df.reset_index(inplace=True, drop=True)

In [ ]:
df.head()

,content
0,나 중간고사 반에서 1등했어
1,중간고사 점수 내가 반에서 제일 잘 받음
2,나 반에서 중간 성적 제일 좋아
3,우리 반에서 내가 시험 제일 잘 봤다
4,중간고사 반 1등 먹음


In [ ]:
df['content'] = df.content.swifter.apply(lambda x: re.sub(r'[^\uAC00-\uD7A30-9a-zA-Z\s]', '', x))

Pandas Apply:   0%|          | 0/10317 [00:00<?, ?it/s]

In [ ]:
print(f"df : {df.head(10)}")

df :                   content
0         나 중간고사 반에서 1등했어
1  중간고사 점수 내가 반에서 제일 잘 받음
2       나 반에서 중간 성적 제일 좋아
3    우리 반에서 내가 시험 제일 잘 봤다
4            중간고사 반 1등 먹음
5          중간시험 내가 반 1등이야
6            너 왜 이렇게 멍청하냐
7             이 새끼 빡대가리잖아
8            너 말귀 못 알아듣냐고
9              병신 이해력 딸리네


# Soynlp

##### WordExtractor
새로운 단어가 나올 때 미등록 단어 문제가 발생하는 것을 해결하기 위한 통계기반 단어 학습 기법

In [ ]:
from soynlp.word import WordExtractor

In [ ]:
word_extractor = WordExtractor(
    min_frequency=10,
    min_cohesion_forward=0.05, 
    min_right_branching_entropy=0.0
)

In [ ]:
sentence = df.content.tolist()
sentence[:10]

['나 중간고사 반에서 1등했어',
 '중간고사 점수 내가 반에서 제일 잘 받음',
 '나 반에서 중간 성적 제일 좋아',
 '우리 반에서 내가 시험 제일 잘 봤다',
 '중간고사 반 1등 먹음',
 '중간시험 내가 반 1등이야',
 '너 왜 이렇게 멍청하냐',
 '이 새끼 빡대가리잖아',
 '너 말귀 못 알아듣냐고',
 '병신 이해력 딸리네']

In [ ]:
len(sentence)

10317

In [ ]:
word_extractor.train(sentence)  # list of str or like
word_extractor.save(cfg.soynlp)    # 모델 저장

words = word_extractor.extract()

training was done. used memory 0.322 Gb
all cohesion probabilities was computed. # words = 1856
all branching entropies was computed # words = 11743
all accessor variety was computed # words = 11743


In [ ]:
words['중간고사']

Scores(cohesion_forward=0.5598113698142824, cohesion_backward=0, left_branching_entropy=1.1972375015579317, right_branching_entropy=1.564957250242801, left_accessor_variety=4, right_accessor_variety=5, leftside_frequency=20, rightside_frequency=0)

##### Noun Extractor
soynlp 명사 추출기

In [ ]:
# from soynlp.noun import LRNounExtractor_v2 #noun extracter v2

In [ ]:
# noun_extractor = LRNounExtractor_v2(verbose=True)
# nouns = noun_extractor.train_extract(sentence)

In [ ]:
# print(list((nouns.keys()))[:150])

##### LTokenizer

In [ ]:
import swifter

In [ ]:
from soynlp.tokenizer import LTokenizer

In [ ]:
cohesion_score = {word:score.cohesion_forward for word, score in words.items()}

##### add user dict into cohesion score with key, value

In [ ]:
with open(cfg.user_dict, 'r') as f:
  user_dict = f.readlines()

In [ ]:
user_dict = [i.replace('\n', '') for i in user_dict]
print(user_dict)

['과몰입', '과몰입러', '쩝쩝박사', '킹갓제너럴', '플렉스', '내적그루브', '단짠단짠', '앱테크', '국룰', 'ㄹㅇ', '손민수', '박박', '누물보', '알잘딱깔센', '할많하않', '맛도리', '떡상', '돼지런', '이왜진', '다이어터', '아가리어터', '유지어터', '내돈내산', '킹리적 갓심', '돈쭐', '좋못사', '레게노', '어라랍스타', '무물보', '쿠쿠루삥뽕', '크크루삥뽕', '멍청비용', '쫌쫌따리', '캘박', '쭈빠삐무녜뇨', '만반잘부', '억까', '졌잘싸', '얼죽아', '찐텐', 'JMT', '존맛탱', '휘뚤마뚤', '휘뚜루마뚜루', '떡락', '흠터레스팅', '스불재', '탈룰라', '대춘', '5252', '껄껄', '따흐흑', '따흑', '야아', '얌', '오예', '헐', '헤헤', '희희', '꺄르륵', '크킄', '야야야', '워어', '얌마', '흑흑', '크크', '두둥', '레알', '왤케', '존나', '쥰내', '갈기갈기', '싸악', '강쥐', '극혐', '노잼', '등극', '리대춘', '머박', '와따시', '와타시', 'K장녀', '고먐미', '다요트', '머가리', '스타벅스', '별다방', '위시템', '홈트', '킹받', '와르르', '이것', '그것', '저것', '이거', '그거', '저거', '그게', '이게', '쥬륵', '쥬르르', '참나', '퉤', '대가리', '아아메', '정주행', '착장', '허허', '후후', '취저', '인듯', '한듯']


In [ ]:
for i in user_dict:
  cohesion_score[i] = 1.0

In [ ]:
print(cohesion_score['과몰입'])
print(cohesion_score['킹갓제너럴'])

1.0
1.0


##### tokenizer

In [ ]:
tokenizer = LTokenizer(scores=cohesion_score)

In [ ]:
sent = "과몰입 하지마. 내가 중간고사에서 점수 제일 잘 받았다. 킹갓제네럴. 내적그루브 뿜뿜"

In [ ]:
print(tokenizer.tokenize(sent))

['과몰입', '하지마.', '내가', '중간고사', '에서', '점수', '제일', '잘', '받았', '다.', '킹갓제네럴.', '내적그루브', '뿜뿜']


In [ ]:
df['soynlp'] = df.content.swifter.apply(lambda x: tokenizer.tokenize(x))

Pandas Apply:   0%|          | 0/10317 [00:00<?, ?it/s]

In [ ]:
df.head()

,content,soynlp
0,나 중간고사 반에서 1등했어,"[나, 중간고사, 반에, 서, 1등, 했어]"
1,중간고사 점수 내가 반에서 제일 잘 받음,"[중간고사, 점수, 내가, 반에, 서, 제일, 잘, 받음]"
2,나 반에서 중간 성적 제일 좋아,"[나, 반에, 서, 중간, 성적, 제일, 좋아]"
3,우리 반에서 내가 시험 제일 잘 봤다,"[우리, 반에, 서, 내가, 시험, 제일, 잘, 봤다]"
4,중간고사 반 1등 먹음,"[중간고사, 반, 1등, 먹음]"


##### 불용어 제거

In [ ]:
stopwords = ['은', '는', '이', '가', '을', '를', '그', '에서', '으로', '로서', '로써', '로', '에']

df['soynlp_stop'] = df.soynlp.swifter.apply(lambda x: [i for i in x if i not in stopwords])

Pandas Apply:   0%|          | 0/10317 [00:00<?, ?it/s]

In [ ]:
df.head(10)

,content,soynlp,soynlp_stop
0,나 중간고사 반에서 1등했어,"[나, 중간고사, 반에, 서, 1등, 했어]","[나, 중간고사, 반에, 서, 1등, 했어]"
1,중간고사 점수 내가 반에서 제일 잘 받음,"[중간고사, 점수, 내가, 반에, 서, 제일, 잘, 받음]","[중간고사, 점수, 내가, 반에, 서, 제일, 잘, 받음]"
2,나 반에서 중간 성적 제일 좋아,"[나, 반에, 서, 중간, 성적, 제일, 좋아]","[나, 반에, 서, 중간, 성적, 제일, 좋아]"
3,우리 반에서 내가 시험 제일 잘 봤다,"[우리, 반에, 서, 내가, 시험, 제일, 잘, 봤다]","[우리, 반에, 서, 내가, 시험, 제일, 잘, 봤다]"
4,중간고사 반 1등 먹음,"[중간고사, 반, 1등, 먹음]","[중간고사, 반, 1등, 먹음]"
5,중간시험 내가 반 1등이야,"[중간, 시험, 내가, 반, 1등, 이야]","[중간, 시험, 내가, 반, 1등, 이야]"
6,너 왜 이렇게 멍청하냐,"[너, 왜, 이렇게, 멍청하냐]","[너, 왜, 이렇게, 멍청하냐]"
7,이 새끼 빡대가리잖아,"[이, 새끼, 빡대가리잖아]","[새끼, 빡대가리잖아]"
8,너 말귀 못 알아듣냐고,"[너, 말귀, 못, 알아, 듣냐고]","[너, 말귀, 못, 알아, 듣냐고]"
9,병신 이해력 딸리네,"[병신, 이해력, 딸리네]","[병신, 이해력, 딸리네]"


# Okt

In [ ]:
from konlpy.tag import Okt

In [ ]:
okt = Okt()

In [ ]:
stopwords = ['의','가','이','은','들','는','좀','잘','걍','과','도','를','으로','자','에','와','한','하다']

def make_pos(content):
    result = okt.pos(str(content))

    # 불용어 제거
    result = [word for word in result if not word[0] in stopwords]
    
    # Noun 명사, Verb 동사, Adjective 형용사, Adverb 부사, Exclamation 감탄사, Suffix 접미사, KoreanParticle 신조어, Number 숫자
    # 조사, 관형사, 접속사 등 제거
    result = [i[0] for i in result if i[1] == 'Noun' or i[1] == 'Verb' or i[1] == 'Adjective' or i[1] == 'Adverb' or i[1] == 'Exclamation' or i[1] == 'Suffix' or i[1] == 'KoreanParticle' or i[1] == 'Number']

    return result

In [ ]:
df['okt'] = df.content.swifter.apply(make_pos)

Pandas Apply:   0%|          | 0/10317 [00:00<?, ?it/s]

In [ ]:
df.head()

,content,soynlp,soynlp_stop,okt
0,나 중간고사 반에서 1등했어,"[나, 중간고사, 반에, 서, 1등, 했어]","[나, 중간고사, 반에, 서, 1등, 했어]","[나, 중간고사, 반, 1등]"
1,중간고사 점수 내가 반에서 제일 잘 받음,"[중간고사, 점수, 내가, 반에, 서, 제일, 잘, 받음]","[중간고사, 점수, 내가, 반에, 서, 제일, 잘, 받음]","[중간고사, 점수, 내, 반, 제일, 받음]"
2,나 반에서 중간 성적 제일 좋아,"[나, 반에, 서, 중간, 성적, 제일, 좋아]","[나, 반에, 서, 중간, 성적, 제일, 좋아]","[나, 반, 중간, 성적, 제일, 좋아]"
3,우리 반에서 내가 시험 제일 잘 봤다,"[우리, 반에, 서, 내가, 시험, 제일, 잘, 봤다]","[우리, 반에, 서, 내가, 시험, 제일, 잘, 봤다]","[우리, 반, 내, 시험, 제일, 봤다]"
4,중간고사 반 1등 먹음,"[중간고사, 반, 1등, 먹음]","[중간고사, 반, 1등, 먹음]","[중간고사, 반, 1등, 먹음]"


# Save

In [ ]:
df = df[['content', 'okt', 'soynlp_stop']]

In [ ]:
df.rename(columns={'soynlp_stop' : 'soynlp'}, inplace=True)

<ipython-input-41-9a2a8a399dd5>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.rename(columns={'soynlp_stop' : 'soynlp'}, inplace=True)


In [ ]:
df.head()

,content,okt,soynlp
0,나 중간고사 반에서 1등했어,"[나, 중간고사, 반, 1등]","[나, 중간고사, 반에, 서, 1등, 했어]"
1,중간고사 점수 내가 반에서 제일 잘 받음,"[중간고사, 점수, 내, 반, 제일, 받음]","[중간고사, 점수, 내가, 반에, 서, 제일, 잘, 받음]"
2,나 반에서 중간 성적 제일 좋아,"[나, 반, 중간, 성적, 제일, 좋아]","[나, 반에, 서, 중간, 성적, 제일, 좋아]"
3,우리 반에서 내가 시험 제일 잘 봤다,"[우리, 반, 내, 시험, 제일, 봤다]","[우리, 반에, 서, 내가, 시험, 제일, 잘, 봤다]"
4,중간고사 반 1등 먹음,"[중간고사, 반, 1등, 먹음]","[중간고사, 반, 1등, 먹음]"


In [ ]:
fm.save(cfg.temp_data_colab, df)

Saved 10317 records
